## Problem 3.3

In [1]:
from fractions import Fraction
# from copy import deepcopy
from fractions import gcd

def mat_mult(a, b):
    '''matrix multiplication'''
    zip_b = zip(*b)
    return [[sum(a_ij*b_ij for a_ij, b_ij in zip(row_a, col_b))
        for col_b in zip_b] for row_a in a]

# def gcd_my(a, b):
#     '''greatest common divider
#     another way: Fractions.gcd'''
#     while b: a, b = b, a % b
#     return a

def lcm(*l):
    '''least common multiple for any number of numbers'''
    def lcm2(x, y):
        '''least common multiple for 2 numbers'''
        return (max(x,y)/gcd(x,y))*min(x,y) # simplify before multiplying
    return reduce(lambda x, y: lcm2(x, y), l)

def mat_inv_Cramer(m):
    '''matrix inversion, Cramer's rule 
    too long for len(m)>9 (>20min)
    for correct division "/" by dterminant m must consist of Fraction()'''
    
    def mat_transpose(m):
        '''tanspose matrix in place'''
        l , l1 = len(m), len(m[0])
        if l!=l1: raise ValueError('can`t transpose nonsquare matrix')
        for i in xrange(l-1):
            for j in xrange(i+1, l):
                m[i][j], m[j][i] = m[j][i], m[i][j]
    
    def mat_minor(m, i, j):
        '''i, j minor of matrix m'''
        if len(m)==1: raise ValueError('no minors for vector matrix')
        return [row[:j]+row[j+1:] for (ir, row) in enumerate(m) if ir!=i]

    def mat_det(m):
        '''matrix determinant'''
        if len(m)==1: return m[0][0] # stopping creteria
        return sum( ((-1)**j)*m[0][j]*mat_det(mat_minor(m,0,j)) for j in xrange(len(m)) )

    def mat_cofactor(m, i, j):
        '''i, j cofactor of matrix m'''
        return ((-1)**(i+j)) * mat_det( mat_minor(m,i,j) )
    
    # determinant
    det = mat_det(m)
    if det==0: raise ValueError('not inverteble, det==0')
    if len(m)==1 and len(m[0])==1: return [[1/det]]
    # cofactor matrix
    mRange = xrange(len(m))
    c = [[mat_cofactor(m,i,j) for j in mRange] for i in mRange]
    mat_transpose(c)
    # c/d
    for i in mRange:
        for j in  mRange: c[i][j] /= det
    return c

def mat_inv_Gauss(m):
    '''inverse matrix by gauss elimination'''
    # helping functions
    def add_eye(m):
        '''adds identity matrix to the right of m
        changes in place'''
        nx = xrange(len(m))
        for i, row in enumerate(m):
            row += [Fraction(int(i==j),1) for j in nx]

    def norm_row_by_elem(m, i, j):
        '''makes element j in raw i =1
        all other elements of raw i normalize accordingly
        canges m in place'''
        factor = m[i][j]
        for k in xrange(len(m[0])): m[i][k] /= factor

    def find_row_nonzero_i(m, i):
        '''returns index of 1st row (start search from i)
        with elem on ith place nonzero 
        if none found raise exception as matrix is not invertable
        '''
        nx = xrange(i,len(m))
        for j in nx:
            if m[j][i] != 0: return j
        raise ValueError('noninvertable matrix')

    # def zero_col_j_by_row_i(m, im, jm):
    #     '''use row im to 0 values in column jm by adding row_im multiple
    #     exclude this row, changes m in place'''
    #     row = m[im]
    #     nx = xrange(len(row))
    #     for i in (range(im) + range(im+1,len(m))): # exclude im
    #         factor = m[i][jm]/row[jm]
    #         if factor.numerator==0: continue
    #         for j in nx:
    #             m[i][j] -= row[j] * factor
    def zero_col_i(m, im):
        '''use row im to 0 values in col i below by adding row_im multiple
        exclude this row, the element i,i already 1
        changes m in place'''
        row = m[im]
        nx = xrange(im+1, len(row))
        for i in (range(im) + range(im+1,len(m))): # exclude im
            factor = m[i][im]
            if factor.numerator==0: continue
            for j in nx: m[i][j] -= row[j] * factor
    #========        
    ## main algorithm
    add_eye(m)
    for i in xrange(len(m)):
        ix = find_row_nonzero_i(m, i)
        m[i], m[ix] = m[ix], m[i] #swap_raws
        norm_row_by_elem(m, i, i)
        zero_col_i(m, i)
    return [row[len(m):] for row in m]

def get_IQ_R_matrices(m, trans, absorb):
    '''canonical form
    [Q R]
    [0 I]
    Q - transient probabilities, R probs to go to absorbing states
    extract IQ=(I-Q) and R
    '''
    Q = [[Fraction(m[i][j], sum(m[i])) for j in trans] for i in trans]
    R = [[Fraction(m[i][j], sum(m[i])) for j in absorb] for i in trans]
    # q is square
    nq = xrange(len(Q))
    IQ = [[int(i==j) - Q[i][j] for j in nq] for i in nq]
    return IQ, R

def solution(m):
    '''
    idea: check wiki on Stochastic matrix and Absorbing Markov chain
    easy to proove that top row of (I-Q)^(-1)*R gives answer
    where probability matrix of transitions: P = [[Q, R], [0, I]], Q square matrix, I- stationary states
    need to find stationary states and fill  Q, R matrixes,
    code matrix multiplication, matrix inversion (the main challenge here, 
    e.g. Cramer's rule (simpler code) or gaussian elimnation)
    then for answer need MCM (min common multiple)
    '''
    #find indexes of absorbing (and non-reachible) and transient states
    #need to keep initial order of trans states for correct answer
    # if only one state its 100% prob to stay there
    if len(m)==0: return [1,1]
    trans, absorb = [], []
    for ix, row in enumerate(m):
        if sum(row)==0: absorb.append(ix)
        else: trans.append(ix)
    #if 1 absorbing state: its 100% prob to go there
    if len(absorb)==1: return [1,1]

    IQ, R = get_IQ_R_matrices(m, trans, absorb)
#     IQ_inv = mat_inv_Cramer(IQ) # slow at large n>8 matrix
    IQ_inv = mat_inv_Gauss(IQ) #spoils IQ but we do not need it any more (use deepcopy())
    #probability to drop to stationary absorbing states from state 0
    P0 = mat_mult(IQ_inv,R)[0]
    
    #simplify list of fractions to common denominator
    comonDenMult = lcm(*[frac.denominator for frac in P0])
    #normalize numerators
    ans = [frac.numerator*(comonDenMult/frac.denominator) for frac in P0]
    return ans + [comonDenMult]


In [2]:
def test():
    #format [(n, answ), ...]
    testLists = [
    ([
        [0, 1, 2, 0],
        [0, 0, 0, 0],
        [3, 0, 0, 4],
        [0, 0, 0, 0]
    ], [7, 8, 15]),
    ([
        [0, 1, 0, 0, 0, 1],
        [4, 0, 0, 3, 2, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]
    ], [0, 3, 2, 9, 14]),
    ([
          [0, 2, 1, 0, 0],
          [0, 0, 0, 3, 4],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]
    ],[7, 6, 8, 21]),
    ]
    #testing
    for n, right in testLists:
        answ = solution(n)
#         print(answ)
        if answ!=right: print "error: sol({})->{}, not {}".format(n, answ, right)
            

In [3]:
test()

In [377]:
# from random import randint
# %%time
# n=9 #-> 2min 17sec
# mat_inv_Cramer([[Fraction(randint(-3,3),1) for j in range(n)] for i in range(n)])

# n=10 #-> 32ms
# mat_inv_Gauss([[Fraction(randint(-3,3),1) for j in range(n)] for i in range(n)])

## Idea





check wiki on Stochastic matrix and Absorbing Markov chain
easy to proove that top row of (I-Q)^(-1)*R gives answer
need to code Q, R matrixes,matrix multiplication, matrix inversion (see gaussian elimnation)
then for answer need MCM (min common multiple)

## * description *